<a href="https://colab.research.google.com/github/kingbingodbin/NLP_models/blob/main/XLnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from torchtext.data.utils import get_tokenizer
from torchtext.data import Field
import pandas as pd
import os
import zipfile
from itertools import combinations

In [ ]:
import torchtext
import io

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import os
os.listdir('drive/My Drive/datasets/wikitext-2')

['wiki.valid.tokens', 'wiki.test.tokens', 'wiki.train.tokens']

In [ ]:
#install package for subword tokenizer
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 7.9MB/s 


In [ ]:
#install package for subword tokenizer
!pip install revtok

tokenizer

In [ ]:
import sentencepiece as spm

In [ ]:
import urllib.request
import io
import sentencepiece as spm

# Loads model from URL as iterator and stores the model to BytesIO.
model = io.BytesIO()
with urllib.request.urlopen(
    'https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt'
) as response:
  spm.SentencePieceTrainer.train(
      sentence_iterator=response, model_writer=model, vocab_size=1000)

# Serialize the model as file.
# with open('out.model', 'wb') as f:
#   f.write(model.getvalue())

# Directly load the model from serialized model.
sp = spm.SentencePieceProcessor(model_proto=model.getvalue())
print(sp.encode('this is test'))

[64, 47, 4, 13, 391]


In [ ]:
"""load wikitext2 data"""

'load wikitext2 data'

In [ ]:
tokenizer = get_tokenizer('revtok')

In [ ]:
dataset_base_path = 'drive/My Drive/datasets/'
path = dataset_base_path + 'wikitext-2/'
#use reversable tokenizer for flexible 
TEXT = torchtext.data.ReversibleField(tokenize = get_tokenizer("revtok"),
                            init_token = '<sos>',
                            eos_token = '<eos>',
                            unk_token ='<unk>',
                            sequential=True,
                            lower=True,
                            use_vocab=True)

In [ ]:
dtype = torch.float
device = torch.device("cpu")

In [ ]:
tr_dat,val_dat,te_dat = torchtext.datasets.language_modeling.WikiText2.splits(TEXT)

downloading wikitext-2-v1.zip


wikitext-2-v1.zip: 100%|██████████| 4.48M/4.48M [00:00<00:00, 8.46MB/s]


extracting


In [ ]:
[tr_dat.examples[0].text][0][:10]

[' ',
 ' ',
 '<eos>',
 ' ',
 ' = ',
 ' valkyria ',
 ' chronicles ',
 ' iii ',
 ' = ',
 ' ']

In [ ]:
[val_dat.examples[0].text][0][:10]

[' ', ' ', '<eos>', ' ', ' = ', ' homarus ', ' gammarus ', ' = ', ' ', '<eos>']

In [ ]:
[te_dat.examples[0].text][0][:10]

[' ', ' ', '<eos>', ' ', ' = ', ' robert ', ' <', ' unk ', '> ', ' = ']

In [ ]:
hasattr(TEXT, "vocab")

False

In [ ]:
#seperating wikitext2 data to documents
def sep_to_docs(data : list, doc_sep : str, token_sep : str) -> list:
  """returns documents from split input text chunk
  """
  result = []
  docs = (''.join(data)).split(doc_sep)
  for i in docs:
    curr_doc = (''.join(i)).split(token_sep)
    curr_doc = [i for i in curr_doc if i != '']
    if len(curr_doc) >= 1:
      result.append(curr_doc)
  return result

In [ ]:
tr_docs = sep_to_docs(data = [tr_dat.examples[0].text][0], doc_sep = '<eos>  =', token_sep = '  ')
val_docs = sep_to_docs(data = [val_dat.examples[0].text][0], doc_sep = '<eos>  =', token_sep = '  ')
te_docs = sep_to_docs(data = [te_dat.examples[0].text][0], doc_sep = '<eos>  =', token_sep = '  ')

In [ ]:
print(len(tr_docs),len(val_docs),len(te_docs))

6211 620 708


In [ ]:
#create vocab with glove.6B.100d
#TEXT.build_vocab(tr_docs,min_freq=10,vectors="glove.6B.100d")
#!cp .vector_cache/glove.6B.zip drive/"My Drive"/datasets/


#TEXT.build_vocab(tr_docs,min_freq=10)

In [ ]:
"""extracting vocab"""
#TEXT.build_vocab(tr_docs,min_freq=10,vectors="glove.6B.100d")
#!cp .vector_cache/glove.6B.zip drive/"My Drive"/datasets/
#glove_zip = zipfile.ZipFile('drive/My Drive/datasets/glove.6B.zip')
#glove_zip.extractall('drive/My Drive/datasets/pretrained_wordvectors')
#glove_zip.close()

'extracting vocab'

In [ ]:
from torchtext.vocab import Vectors
glove_6b_300d_vec = Vectors(name='glove.6B.300d.txt', cache='drive/My Drive/datasets/pretrained_wordvectors')

special_tokens = '<cls>'
TEXT.build_vocab(tr_docs, min_freq=10, vectors=glove_6b_300d_vec)

In [ ]:
vocab_size = len(TEXT.vocab.itos)
word_embedding_size = TEXT.vocab.vectors.shape[1]

In [ ]:
#sample embedding
embeds = nn.Embedding(vocab_size, word_embedding_size)
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

NameError: ignored

In [ ]:
embeds = nn.Embedding(vocab_size, word_embedding_size)



In [ ]:
aa = TEXT.numericalize([te_docs[0][:10]])

In [ ]:
aa.squeeze(1)

tensor([637,   8,  14,   3,   3, 637,   8,  26,  33, 340])

In [ ]:
TEXT.vocab.vectors[[TEXT.vocab.stoi.get('<sep>')]]

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [ ]:
def build_Field_with_vocab(
    text_dat: list,
    Field: torchtext.data.Field,
    special_tokens: list=None, 
    min_freq: int=10, 
    learned_vectors: Vectors=None):

  if special_tokens is not None:
    text_dat = text_dat + [special_tokens * min_freq]

  if isinstance(learned_vectors, Vectors):
    Field.build_vocab(
        text_dat,
        min_freq=min_freq,
        vectors=learned_vectors
        )
  else:
    Field.build_vocab(text_dat, min_freq=min_freq)
  
  return Field


In [ ]:
TEXT = build_Field_with_vocab(text_dat=tr_docs, Field=TEXT, special_tokens =['<cls>', '<sep>'], min_freq=10, learned_vectors=glove_6b_300d_vec)

In [ ]:
TEXT

NameError: ignored

In [ ]:
class Xlnet_Input(torch.utils.data.Dataset):
  """Creates input data with unlabeled text data
   input : list of unlabeled text in list form
   output : dataset, list of tuple(Text, seq_len, context, relative_position, segment)"""
  def __init__(self, text: list, Field: Field, do_perm: bool=True, sep_docs: bool=True, doc_sep: str='<eos>  =', token_sep : str='  '):
    """text_dat: list of unlabeled documents(list of lists), should be tokenized with 'revtok'
    """
    #dict
    #context_encoding, segment_encoding
    #positional encoding

    self.doc_sep = doc_sep
    self.token_sep = token_sep
    self.do_perm = do_perm
    self.Field = Field

    if sep_docs:
      text = self.sep_to_docs(text, doc_sep, token_sep)

    
    self.text_dat =[]
    for doc in text:
      self.text_dat.append(self.Field.numericalize([doc]))
  

  def create_xlnet_input(self,segment_len: int, reuse_len: int, seq_len: int, d_model: int=256, fixed_length_segmentation: bool=True, random_segment_ratio: float=0.5, min_pred_len: int=5):

    """creates input data sample for xlnet,
    create all dataset
    __getitem__ """

    #self.segment_len = segment_len
    self.d_model = d_model
    self.min_pred_len = min_pred_len

    if fixed_length_segmentation:
      self.segmentation(segment_len, reuse_len)

    sep_token = self.Field.vocab.stoi.get('<sep>')

    cls_token = self.Field.vocab.stoi.get('<cls>')

    #context, if doc

    self.sampled_segment_pair = self.sample_segments(random_segment_ratio)

    self.input = []

    self.create_permed_masked_matrix(min_pred_len)

    for first_idx,second_idx in self.sampled_segment_pair:
        #self.permutation, masking을 둘다해줘야함
      
      self.input.append(
          [
            torch.cat(
                (
                    self.reuse_segment[first_idx],
                    self.Field.vocab.vectors[[cls_token]],
                    self.text_segment[first_idx], 
                    self.Field.vocab.vectors[[sep_token]],
                    self.text_segment[second_idx],
                    self.Field.vocab.vectors[[sep_token]]
                ),
            ),
            torch.cat(
                (
                    self.Field.vocab.vectors[[cls_token]],
                    self.target_text[first_idx], 
                    self.Field.vocab.vectors[[sep_token]],
                    self.target_text[second_idx],
                    self.Field.vocab.vectors[[sep_token]]
                ),
            ),

            (
                len(self.reuse_segment[first_idx]),
                len(self.text_segment[first_idx]),
                len(self.text_segment[second_idx])
              ),
            (
                self.position_segment[first_idx],
                self.position_segment[second_idx]
              ),
            (
                first_idx,
                second_idx
              ),
            (
                self.doc_index[first_idx],
                self.doc_index[second_idx]
              ),
            (
                self.q_stream_perm_mask[first_idx]
                self.c_stream_perm_mask[first_idx]
                self.output_perm_mask[first_idx]
            )
            ]
          )

    
    
    self.input = tuple(self.input)
    
  def segmentation(self, segment_len, reuse_len):
    """split input text into segments
    inputs 
      segment_len : length to which input text is splitted
    
    creates
      text_segment : segmented text data
      position_segment : positional information of text segments
      segment_index : index of segment
      doc_index : index of the documnet(or context)
    """
    self.segment_len = segment_len
    self.reuse_len = reuse_len

    self.text_segment = []
    self.target_text = []
    self.reuse_segment = []
    self.position_segment = []
    self.segment_index = []
    self.doc_index = []
    
    curr_doc_index = 0

    for curr_doc in self.text_dat:

      curr_segment_index = list(range(len(curr_doc) // self.segment_len + 1))
      
      self.segment_index += curr_segment_index

      self.doc_index += [curr_doc_index] * len(curr_segment_index)

      curr_doc_index += 1


      for curr_index in curr_segment_index[:-1]:
        
        self.text_segment.append(
             self.Field.vocab.vectors[
                                curr_doc[
                                              (self.segment_len * curr_index): (self.segment_len * (curr_index+1))
                                              ]
                                ].squeeze(1) 
            )
        self.target_text.append(
             self.Field.vocab.vectors[
                                      curr_doc[
                                               (self.segment_len * curr_index + 1): (self.segment_len * (curr_index+1) + 1)     
                                      ]
             ].squeeze(1)
        )
        self.reuse_segment.append(
             self.Field.vocab.vectors[
                                     curr_doc[
                                                   ((self.segment_len * curr_index) - reuse_len):(self.segment_len * curr_index)
                                                   ]
                                ].squeeze(1)
            )
        
        last_segment_len = len(self.text_segment[-1])
        
        self.position_segment.append(
            torch.tensor(
                range(
                    self.segment_len * curr_index, self.segment_len * curr_index + last_segment_len
                    )
                )
            )


  def sample_segments(self, random_segment_ratio):
    """make segment pair for input data : ["<cls>","segmentA","<sep>","segmentB","<sep>"]
    input:
      random_segment_ratio : probability of choosing random segment instead of real next segment
    output:
      segment_pair: segment pair used in input data creation
    """
    segment_pair = []
    
    n_segments = len(self.text_segment)

    for i in range(n_segments-1):
      if random_segment_ratio < np.random.uniform(size=1):
        segment_pair.append(
            (
                i,
                np.random.randint(0,n_segments-1),
             )
            )
      else:
        segment_pair.append(
            (
                i,i+1
            )
        )

    return tuple(segment_pair)
    
  def sep_to_docs(self, text, doc_sep, tokent_sep) -> list:
  #def sep_to_docs(data : list, doc_sep : str, token_sep : str) -> list:
    """returns documents from split input text chunk
    """
    seperated_docs = []
    docs = (''.join(text)).split(doc_sep)
    for i in docs:
      curr_doc = (''.join(i)).split(token_sep)
      curr_doc = [i for i in curr_doc if i != '']
      if len(curr_doc) >= 1:
        seperated_docs.append(curr_doc)
    return seperated_docs


  def create_permed_masked_input(self, min_pred_len: int=10):
    for first_idx, second_idx in self.sampled_segment_pair:
      input_len = len(self.text_segment[first_idx]) + len(self.text_segment[second_idx]) + 3

      perm_mask = np.zeros((input_len, input_len))

      self.perm_order = np.arange(input_len)

      np.random.shuffle(self.perm_order)

      c_stream_perm_mask = perm_mask

      q_stream_perm_mask = perm_mask

      output_perm_mask = perm_mask

      count = 0
      for i in self.perm_order[input_len - min_pred_len:]:
        c_stream_perm_mask[count:, i] = 1
        count += 1
      
      c_stream_perm_mask[:,self.perm_order[:,:input_len - min_pred_len]] = 1

      count = 0
      for i in self.perm_order[input_len - min_pred_len - 1:-1]:
        q_stream_perm_mask[count:, i] = 1
        count += 1
      
      q_stream_perm_mask[:,self.perm_order[:,:input_len - min_pred_len - 1]] = 1
      

      for mask_until in range(min_pred_len - 1, curr_reuse_len + curr_first_seg_len + curr_second_seg_len + 3)



  def __getitem__(self, idx):
    return self.input[idx]

      #sample = {'image': image, 'landmarks': landmarks}

  def __len__(self):
    return len(self.input)

In [ ]:
aaa = Xlnet_Input([[te_dat.examples[0].text][0]], Field = TEXT, sep_docs=False)

In [ ]:
aaa.create_xlnet_input(128,128,259)

In [ ]:
len(aaa.input[1])

6

In [ ]:
aaa.input[1]

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 (128, 128, 128),
 (tensor([128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,
          142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
          156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
          170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
          184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197,
          198, 199, 200, 201, 202, 203, 204, 205, 206, 207

In [ ]:
dd = []
dd.append([0] * 10)

In [ ]:
[0,1,2][:-1]

[0, 1]

In [ ]:
aaa.Field.vocab.vectors[aaa.Field.vocab.stoi.get("<sep>")]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
aaa.target_text[0:1]

[tensor([[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]])]

In [ ]:
aaa.text_segment[1]

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [ ]:
aaa.target_text[3].shape

torch.Size([5, 300])

In [ ]:
aaa.text_segment[3].shape

torch.Size([6, 300])

In [ ]:
self.target_csv = pd.read_csv(traget_csv)
    self.dat_shape = data.shape
    self.root_dir = root_dir
    self.segment_len = segment_len
    self.word_dict = False
    self.data = data

    if do_shuffle:
      shuffle()
    
    segmentation()

  def segmentation(self):
    self.segments = 

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    doc_paths = os.path.join(self.root_dir, self.target_csv.iloc[idx, 0])
    for doc_path in doc_paths
    doc_file = open(doc_path, 'r')

    lines = doc_file.readlines()

    if word_dict:
      for line in lines:
        for 

    


    doc_file.close()


    f = open("C:/doit/새파일.txt", 'r')


      landmarks = self.landmarks_frame.iloc[idx, 1:]
      landmarks = np.array([landmarks])
      landmarks = landmarks.astype('float').reshape(-1, 2)
      sample = {'image': image, 'landmarks': landmarks}

    return sample

  def __len__(self):
    return len(self.d)

'<eos>  ='

In [ ]:
aa = [tr_dat.examples[0].text][0][:1000]

In [ ]:
result = []
docs = (''.join(aa)).split('<eos>  =')
for i in temp:
  curr_doc = (''.join(i)).split('  ')
  curr_doc = [i for i in curr_doc if i != '']
  result.append(curr_doc)

In [ ]:
train_data[0]

tensor([    7,    29,  4031,  2997,   447,  3285,     4,   133,   444,    36,
         3745,     3,   838,     6, 24073,  1484,     4,    25, 13603,    71])

In [ ]:
train_data[1]

tensor([    7,  1781,    20,   179, 15130,   765,   632,    59,   490,  6997,
           27,     7,     6,     4,   480,  5051,    60,    15,     8,     9])

<bound method Field.build_vocab of <torchtext.data.field.Field object at 0x7f1ca121d400>>

In [ ]:
en_textfield.build_vocab(tr_dat, min_freq = 2)

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (tr_dat, val_dat, test_dat),
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
bb

(tensor([[   9,   11, 3875,  ...,    4,    9,    9]]), tensor([2088628]))

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
ds = tfds.load('glue', split='train', as_supervised=False, shuffle_files=False)

In [ ]:
for i in ds:
  print(i)

NameError: ignored

In [ ]:
aa.load()

AttributeError: ignored

In [ ]:
np.array([1,2,3]).shape()

TypeError: ignored

In [ ]:
ddd = aa(a = np.array([1,2,4,5]),b=np.array([1,2,4,5]))

In [ ]:
exec('b=ddd.a.sum()')

In [ ]:
len(np.array([1,2,3]))

3

In [ ]:
sample_dict.get('aa')

12

In [ ]:
sample_dict = {'aa': 12, 'bb':13}

In [ ]:
'list is list'.split(' ')

['list', 'is', 'list']

In [ ]:
dtype = torch.float
device = torch.device("cpu")

In [ ]:
#활용된 잡지식

#pytorch dataset 불러오기 
#https://pytorch.org/text/datasets.html

#자동으로 추가해야할 내용들
#To avoid clutter, we omit the implementation details including multi-head attention, residual connection, layer normalization and position-wise feed-forward as used in Transformer(-XL). The details are included in Appendix A.2 for reference

#데이터 로딩
#data loading
# https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html

#함수 오버라이딩 방법
#super(MyModel, self).__init__()

#einstein summation
#https://rockt.github.io/2018/04/30/einsum

#defining method with string
#https://stackoverflow.com/questions/11553721/using-a-string-variable-as-a-variable-name

#kwargs,args
#https://brunch.co.kr/@princox/180

In [ ]:
# Two-stream attention cell
#input : q_size, k_size, v_size, segment_index, attention_size, heads =8, **kwargs = {init_Qstream_q_LT_weight, init_Qstream_q_K_weight,init_Qstream_q_R_weight, init_Qstream_k_E_LT_weight, init_Qstream_k_R_LT_weight, init_Qstream_v_LT_weight,
#                                                                            init_Qstream_segment_same_weight, init_Qstream_segment_notsame_weight, init_Qstream_segment_bias, init_Qstream_output_weight,
#                                                                            init_Cstream_q_LT_weight, init_Cstream_q_K_weight,init_Cstream_q_R_weight, init_Cstream_k_E_LT_weight, init_Cstream_k_R_LT_weight, init_Cstream_v_LT_weight,
#                                                                            init_Cstream_segment_same_weight, init_Cstream_segment_notsame_weight, init_Cstream_segment_bias, init_Cstream_output_weight}
#

#operation : q_LT, k_LT, v_LT, segmentation, permutation(optional), masking, relative_position_encoding, segment_encoding, cal_content_attention, cal_query_attention, memorise

#properties : Qstream_q_LT_weight, Qstream_q_K_weight,Qstream_q_R_weight, Qstream_k_E_LT_weight, Qstream_k_R_LT_weight, Qstream_v_LT_weight,
#             Qstream_segment_same_weight, Qstream_segment_notsame_weight, Qstream_segment_bias, Qstream_output_weight,
#             Cstream_q_LT_weight, Cstream_q_K_weight,Cstream_q_R_weight, Cstream_k_E_LT_weight, Cstream_k_R_LT_weight, Cstream_v_LT_weight,
#             Cstream_segment_same_weight, Cstream_segment_notsame_weight, Cstream_segment_bias, Cstream_output_weight content_attention, query_attention, content_memory, query_memory

#output : Dq vector(s)



In [ ]:
class create_PLM_input(data_utils.Dataset):
  def __init__(self, data, target_csv, word_dict = False, sep_by_lines = False, segment_len = False, do_shuffle=True):
    super(create_PLM_input, self).__init__()
    
    

In [ ]:
class create_PLM_input_inmemory(data_utils.Dataset):


In [ ]:
# https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html

perred


In [ ]:
a= torch.nn

In [ ]:
def shuffle()

def permutate()

def rel_position_encoding():

def masking():

In [ ]:
if not a:
  print(1)

In [ ]:
class Two_stream_attention(nn):
  def __init__(self, q_size, k_size, v_size, attention_size = False, heads =8, **kwargs):
    super(Two_stream_attention, self).__init__() 
      

    self.segment_index = segment_index

    #initialize params
    self.initialize_params(self, q_size, k_size, v_size, attention_size, heads)

  def initialize_params(self, q_size, k_size, v_size, attention_size, heads):
    self.Qstream_q_LT_weight = nn.Linear()
    self.Qstream_q_K_weight = 
    self.Qstream_q_R_weight = 
    
    self.Qstream_k_E_LT_weight = 
    self.Qstream_k_R_LT_weight = 
    self.Qstream_v_LT_weight = 
    
    self.Qstream_segment_same_weight = 
    self.Qstream_segment_notsame_weight = 
    self.Qstream_segment_bias = 
    
    self.Qstream_output_weight = 

    self.Cstream_q_LT_weight = 
    self.Cstream_q_K_weight = 
    self.Cstream_q_R_weight = 
    
    self.Cstream_k_E_LT_weight = 
    self.Cstream_k_R_LT_weight = 
    self.Cstream_v_LT_weight = 

    self.Cstream_segment_same_weight = 
    self.Cstream_segment_notsame_weight = 
    self.Cstream_segment_bias = 
    
    self.Cstream_output_weight =

  def set_params(self,**kwargs):
    for i,j in kwargs.items():
      setattr(self, i, j)
      explicitly_defined_vars.append(i)

  def forward(self, input, do_parm = False, rel_position = False):
    
    #record position
    if not rel_position:
      self.rel_position = rel_position_encoding(input)
    else:
      self.rel_position = rel_position

    #permutation
    if do_parm:
      self.permutate()

    #get position encoding

    #cal
  def backward(self,??): #클레스 정의하는 법 참조


SyntaxError: ignored

In [ ]:
_#http://nlp.seas.harvard.edu/2018/04/03/attention.html

In [ ]:
##Copyright 2013, Youngbin Jang, All rights reserved.

In [ ]:
#https://tutorials.pytorch.kr/beginner/nn_tutorial.html